In [4]:
from keras.layers import Bidirectional , GRU ,LSTM
import numpy as np
import pandas as pd


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

from keras.layers import Embedding

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding , Convolution2D
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras.models import Sequential

import numpy as np


In [5]:
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from keras.layers import Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
import theano.ifelse
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from sklearn.model_selection import StratifiedKFold

## Loading the data

In [43]:
train_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/train_medical.csv')
test_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/test_medical.csv')
validation_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/validation_medical.csv')

In [44]:
#frames = [train_data , validation_data]
#train_data = pd.concat(frames)

In [45]:
#train_data.shape

In [46]:
columns_to_drop = ['_unit_id', '_created_at', '_canary', '_id', '_started_at', '_channel',
       '_trust', '_worker_id', '_country', '_region', '_city', '_ip',
       'direction', 'b1', 'b2', 'direction_gold', 'e1', 'e2',
       'relex_relcos', 'sent_id', 'twrex', 'term1' , 'term2']


train_data = train_data.drop( columns_to_drop , axis = 1)
test_data = test_data.drop(columns_to_drop , axis = 1)
validation_data = validation_data.drop(columns_to_drop , axis = 1)

train_data.loc[train_data.relation == 'is location of' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'diagnosed by' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'contraindicates' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'location' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'location of' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'is diagnosed by' , 'relation'] = 'Others'


test_data.loc[test_data.relation == 'is location of' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'diagnosed by' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'contraindicates' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'location' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'location of' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'is diagnosed by' , 'relation'] = 'Others'

validation_data.loc[validation_data.relation == 'is location of' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'diagnosed by' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'contraindicates' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'location' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'location of' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'is diagnosed by' , 'relation'] = 'Others'


In [47]:
print(train_data.shape , test_data.shape , validation_data.shape )

(13340, 2) (4566, 2) (4270, 2)


## Preprocessing with keras


In [48]:
import re
import numpy as np

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')

def text_prepare(doc):
    
    doc = doc.lower()
    doc = REPLACE_BY_SPACE_RE.sub(' ' , doc)
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [49]:
train_data_clean = [ text_prepare(x) for x in train_data.sentence]
validation_data_clean = [ text_prepare(x) for x in validation_data.sentence]

In [50]:
test_data_clean = [ text_prepare(x) for x in test_data.sentence]

### Tokenising 

In [51]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data_clean)

In [52]:
vocab_size = len(tokenizer.word_index) + 1

In [53]:
vocab_size

7835

In [54]:
encoded_docs_train = tokenizer.texts_to_sequences(train_data_clean)
encoded_docs_validation = tokenizer.texts_to_sequences(validation_data_clean)
encoded_docs_test = tokenizer.texts_to_sequences(test_data_clean)


In [56]:
# calculate max document length
lengths = [len(s.split()) for s in train_data_clean]
max_length = max(lengths)
print(max_length)

77


In [57]:
##########padding
max_length = 77
padded_docs_train = pad_sequences(encoded_docs_train, maxlen = max_length, padding='post')
padded_docs_validation = pad_sequences(encoded_docs_validation, maxlen = max_length, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen = max_length, padding='post')


In [59]:
X_train = padded_docs_train
X_val = padded_docs_validation
X_test = padded_docs_test

In [60]:
print( X_train.shape , X_test.shape , X_val.shape )

(13340, 77) (4566, 77) (4270, 77)


In [61]:
encoder = LabelEncoder()
encoder.fit(train_data['relation'])
y_train = encoder.transform(train_data['relation'])
y_val = encoder.transform(validation_data['relation'])
y_test = encoder.transform(test_data['relation'])


#y_train_cat = to_categorical(y_train)
#y_test_cat = to_categorical(y_test)

print(y_train.shape , y_test.shape , y_val.shape)

(13340,) (4566,) (4270,)


## word embeddings

In [62]:
filename_pmc = 'C:/Users/hp/Word_embeddings/PubMed-and-PMC-w2v.bin'
model_pmc = gensim.models.KeyedVectors.load_word2vec_format( filename_pmc  , binary=True)

In [63]:
EMBEDDING_DIM_PMC = 200
vocabulary_size =  vocab_size

In [64]:
embedding_matrix_pmc = np.zeros(( vocabulary_size , EMBEDDING_DIM_PMC ))
for word , i in tokenizer.word_index.items():
    try:
        embedding_vector_pmc = model_pmc[word]
    except KeyError:
        embedding_vector_pmc = None
    if embedding_vector_pmc is not None:
        embedding_matrix_pmc[i] = embedding_vector_pmc
        

## model ( Attention over GRU)

In [65]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)



class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
        
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [66]:
def precision(y_true, y_pred):
    """Precision metric.
     Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    """Recall metric.
     Only computes a batch-wise average of recall.
     Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [70]:
Y_train_cat = to_categorical(y_train)
Y_train_cat.shape

(13340, 3)

In [72]:
Y_val_cat = to_categorical(y_val)
Y_val_cat.shape

(4270, 3)

In [71]:
y_test_cat = to_categorical(y_test) 
y_test_cat.shape

(4566, 3)

In [75]:
model = Sequential()
model.add(Embedding( vocab_size, 200 ,
          weights=[embedding_matrix_pmc], input_length= 77, trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(GRU( 100 ,return_sequences= True, dropout=0.3, recurrent_dropout=0.25)))
model.add(Attention())
model.add(Dense(3, activation="softmax"))

In [76]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc' , precision , recall])

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 77, 200)           1567000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 77, 200)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 77, 200)           180600    
_________________________________________________________________
attention_3 (Attention)      (None, 200)               277       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 603       
Total params: 1,748,480.0
Trainable params: 181,480.0
Non-trainable params: 1,567,000.0
_________________________________________________________________


In [78]:
model.fit(X_train , Y_train_cat , epochs = 10, batch_size = 1000 ,validation_data = ( X_val ,Y_val_cat ) )

Train on 13340 samples, validate on 4270 samples
Epoch 1/10
13340/13340 [==============================] - 97s - loss: 0.9420 - acc: 0.5981 - precision: 0.6434 - recall: 0.1735 - val_loss: 0.8582 - val_acc: 0.6302 - val_precision: 0.6455 - val_recall: 0.4761
Epoch 2/10
13340/13340 [==============================] - 98s - loss: 0.7144 - acc: 0.7355 - precision: 0.7713 - recall: 0.6752 - val_loss: 0.7320 - val_acc: 0.7166 - val_precision: 0.7332 - val_recall: 0.6686
Epoch 3/10
13340/13340 [==============================] - 98s - loss: 0.6257 - acc: 0.7588 - precision: 0.7890 - recall: 0.7136 - val_loss: 0.6911 - val_acc: 0.7101 - val_precision: 0.7493 - val_recall: 0.6489
Epoch 4/10
13340/13340 [==============================] - 99s - loss: 0.5895 - acc: 0.7698 - precision: 0.8046 - recall: 0.7214 - val_loss: 0.6586 - val_acc: 0.7152 - val_precision: 0.7603 - val_recall: 0.6726
Epoch 5/10
13340/13340 [==============================] - 101s - loss: 0.5642 - acc: 0.7780 - precision: 0.8075

In [79]:
scores = model.evaluate(X_test, y_test_cat)
print("%s : %.2f%%" % (model.metrics_names[1] , scores[1]*100 ))

4566/4566 [==============================] - 11s    
acc : 73.00%


In [80]:
print("%s : %.2f%%" % (model.metrics_names[2] , scores[2]*100 ))

precision : 75.35%


In [81]:
print("%s : %.2f%%" % (model.metrics_names[3] , scores[3]*100 ))

recall : 69.25%


In [82]:
precision = scores[2]*100
recall = scores[3]*100
## f1 score
f1_score = (2 * precision * recall )/( recall + precision )


In [83]:
f1_score

72.1724162566137